In [ ]:
from pypdf import PdfReader
import re
import os
import glob

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

def extract_activities_from_text(text, year, month):
    pattern = r"^(\d{2}\/\d{2})\s+(.*?)\s+((-?\d{1,3}(,\d{3})*)*\.\d{2})"

    lines = []
    for line in text.split("\n"):
        match = re.match(pattern,line) 
        if match:
            if match.group(1)[:2] == "12" and month == "01":
                this_year = str(int(year) - 1)
            else:
                this_year = year
            lines.append(f"{match.group(1)}/{this_year},{match.group(2)},{match.group(3).replace(',','')}")
    return "\n".join(lines)

directory_path = "/Users/kingwanlau/Downloads/spending/citi"
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
activities = "Transaction date,Description,Amount"
for pdf in pdf_files:
    year = os.path.basename(pdf).split("-")[0][:4]
    month = os.path.basename(pdf).split("-")[0][4:6]
    text_data = extract_text_from_pdf(pdf)
    # activities += "\n" + extract_activities_from_text(text_data, year, month)
    activities += text_data

with open("citi-pdf.csv", "w") as file:
    file.write(activities)



In [ ]:
import pdfplumber
import re
import os
import glob

directory_path = "/mnt/c/users/lauki/OneDrive - Blackstone/Downloads/citi"
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
activities = []
for pdf_file in pdf_files:
    month = os.path.basename(pdf_file).split(" ")[0]
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            activities += re.findall(r"(\d{2}\/\d{2})\s+\d{2}\/\d{2}\s+(.*?)\s+-?\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+(?:\.\d{2})?)", text)


# match = re.findall(r"\s+-?\$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+(?:\.\d{2})?)", text)

print(activities)

In [ ]:
import pdfplumber
import re
import os
import glob

directory_path = "/Users/kingwanlau/Downloads/spending/citi"
pdf_files = glob.glob(os.path.join(directory_path, "Jul 2023.pdf"))
activities = []
text = ""
# activities = [('Transaction Date','Description','Amount')]
for pdf_file in pdf_files:
    month, year = os.path.basename(pdf_file).split(".")[0].split(" ")
    this_activites = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text+= page.extract_text()

with open("text.txt", "w") as file:
    file.write(text)

In [ ]:
# /(\d{2}\/\d{2})\s+\d{2}\/\d{2}\s+(.*?)\s+(-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+(?:\.\d{2})?)(\s\d+\%.*?)*/gm
# /(\d{2}\/\d{2})\s+\d{2}\/\d{2}\s+(.*?)\s+(-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?)/gm
# /(\d{2}\/\d{2})\s+\d{2}\/\d{2}\s+(-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?)/gm
# /(\d{2}\/\d{2})\s+\d{2}\/\d{2}(\s+.*?\s+|\s)(-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?)/gm

In [ ]:
import pdfplumber
import re
import os
import glob

directory_path = "/Users/kingwanlau/Downloads/spending/citi"
pdf_files = glob.glob(os.path.join(directory_path, "*.pdf"))
activities = []
# activities = [('Transaction Date','Description','Amount')]
for pdf_file in pdf_files:
    month, year = os.path.basename(pdf_file).split(".")[0].split(" ")
    this_activites = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            this_activites += re.findall(r"(\d{2}\/\d{2})\s+\d{2}\/\d{2}\s+(.*?)\s+(-?\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?|\d+(?:\.\d{2})?)\n", text)
    for i, activity in enumerate(this_activites):
        txn_date = activity[0] + "/" + (str(int(year)-1) if activity[0][:2] == "12" and month == "01" else year)
        this_activites[i] = (txn_date, activity[1], activity[2].replace("$",""))  # Update the tuple at index i
    activities += this_activites


In [ ]:
month, year = os.path.basename(pdf_file).split(".")[0].split(" ")

activities

In [ ]:
txn_date

In [ ]:
import pandas as pd
import io

df = pd.DataFrame(activities, columns=['Transaction Date','Description','Amount'])
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], format="%m/%d/%Y")
df = df.sort_values(by="Transaction Date", ascending=False)

df.to_csv("citi-pdf2.csv", index=False, header=False)

